In [1]:

from termcolor import colored
from IPython.display import Markdown, display

import bs4
from langchain import hub
from langchain.text_splitter import RecursiveCharacterTextSplitter
from langchain_community.document_loaders import WebBaseLoader, SeleniumURLLoader, PDFPlumberLoader
from langchain_community.vectorstores import Chroma
from langchain_core.output_parsers import StrOutputParser
from langchain_core.runnables import RunnablePassthrough, RunnableParallel
from langchain_openai import ChatOpenAI, OpenAIEmbeddings

import os
from dotenv import load_dotenv
load_dotenv()

True

In [2]:
openai_key = os.getenv("OPENAI_API_KEY")

In [3]:
def printmd(string):
    display(Markdown(string))

# RAG

## Loading web news data to the database

In [4]:
urls = ["https://pt.wikipedia.org/wiki/Javier_Milei",
        "https://www.cbsnews.com/news/super-bowl-winners-list-history/",
        "https://ge.globo.com/futebol/times/corinthians/noticia/2024/02/16/corinthians-anuncia-a-contratacao-do-lateral-direito-matheuzinho.ghtml",
        "https://ge.globo.com/futebol/times/botafogo/noticia/2024/02/16/luiz-henrique-do-botafogo-tem-lesao-na-panturrilha.ghtml",
        "https://ge.globo.com/futebol/times/fluminense/noticia/2024/02/16/escalacao-do-fluminense-diniz-escala-reservas-contra-madureira-mas-tera-john-kennedy-e-douglas-costa.ghtml",
    "https://www.lemonde.fr/football/article/2024/02/25/ligue-1-le-psg-arrache-le-nul-de-justesse-contre-rennes-monaco-remonte-sur-le-podium_6218528_1616938.html",
    "https://ge.globo.com/futebol/futebol-internacional/futebol-frances/copa-da-franca/noticia/2024/02/27/perri-e-heroi-nos-penaltis-e-lyon-avanca-de-fase-na-copa-da-franca.ghtml",
    "https://ge.globo.com/tenis/noticia/2024/02/27/joao-fonseca-sofre-com-condicoes-ruins-da-quadra-no-atp-santiago-e-e-eliminado.ghtml",
    
]
loader = SeleniumURLLoader(urls=urls)
docs = loader.load()

## Loading text from PDF using PyPDFLoader

In [16]:
loader = PDFPlumberLoader("./pdfs/INSPECTION_REPORT.pdf")
docs = loader.load()

In [17]:
for idx, doc in enumerate(docs):
    # add project metadata
    doc.metadata["project"] = "Ghana Early Phase Gas Infrastructure Project"
    doc.metadata["ref"] = "JPP/HF-12/048"
    doc.metadata["title"] = "INSPECTION REPORT Nr LNP/BJ-12/048-CD01"
    doc.metadata["client"] = "Sinopec International Petroleum Service Company (SIPSC)"
    doc.metadata["Manufacturer"] = "SICHUAN KEYCON VALVE MANUFACTURING CO., LTD (KCON)"
    doc.metadata["job_nr"] = "LNP/BJ-12/048"
    doc.metadata["inspection_date"] = "2013-1-27"
    doc.metadata["inspection_location"] = "Guanghan city, P. R. China"
    doc.metadata["issue_date"] = "2013-2-3"
    
    # delete metadata
    #del doc.metadata["Title"]
    
docs[7].metadata

{'source': './pdfs/INSPECTION_REPORT.pdf',
 'file_path': './pdfs/INSPECTION_REPORT.pdf',
 'page': 7,
 'total_pages': 8,
 'Producer': 'GPL Ghostscript 9.26',
 'CreationDate': "D:20190417033156-07'00'",
 'ModDate': "D:20190417033156-07'00'",
 'Author': '',
 'Creator': '',
 'Title': '',
 'Subject': '',
 'Keywords': '',
 'project': 'Ghana Early Phase Gas Infrastructure Project',
 'ref': 'JPP/HF-12/048',
 'title': 'INSPECTION REPORT Nr LNP/BJ-12/048-CD01',
 'client': 'Sinopec International Petroleum Service Company (SIPSC)',
 'Manufacturer': 'SICHUAN KEYCON VALVE MANUFACTURING CO., LTD (KCON)',
 'job_nr': 'LNP/BJ-12/048',
 'inspection_date': '2013-1-27',
 'inspection_location': 'Guanghan city, P. R. China',
 'issue_date': '2013-2-3'}

In [5]:
# Splitting the documents
text_splitter = RecursiveCharacterTextSplitter(chunk_size=1000, chunk_overlap=100)
splits = text_splitter.split_documents(docs)

In [6]:
# save to disk
vectorstore = Chroma.from_documents(documents=splits, embedding=OpenAIEmbeddings(), persist_directory="./chroma_db")

In [3]:
# load from disk
vectorstore = Chroma(persist_directory="./chroma_db", embedding_function=OpenAIEmbeddings())
vectorstore

In [4]:
vectorstore._collection.count()

190

In [41]:
collection = vectorstore.get(
    where = {
        "ref": "JPP/HF-12/048"
    }
)
len(collection['ids'])

11

In [5]:
# RAG Chain
# Retrieve and generate using the relevant snippets of the blog.
retriever = vectorstore.as_retriever()
prompt = hub.pull("rlm/rag-prompt")
llm = ChatOpenAI(model_name="gpt-3.5-turbo", temperature=0)


def format_docs(docs):
    return "\n\n".join(doc.page_content for doc in docs)


rag_chain_from_docs = (
    RunnablePassthrough.assign(context=(lambda x: format_docs(x["context"])))
    | prompt
    | llm
    | StrOutputParser()
)

rag_chain_with_source = RunnableParallel(
    {"context": retriever, "question": RunnablePassthrough()}
).assign(answer=rag_chain_from_docs)

# Multiple News Queries

In [14]:
query = ["Quem é o atual presidente da Argentina?",
         "Quem ganhou o Super Bowl 2024?",
         "Quem é o jogador novo do Corinthians?",
         "Quem é o jogador Botafoguense que se contundiu recenteente?",
         "Qual a formação usada no Fluminense no jogo contra o Madureira?",
         "Quem é o novo reforço do Fluminense?",
         "Qual a nova resulução sobre 'saidinhas' de presos?",
         ]
for idx, q in enumerate(query):
    print(f"Query {idx+1}: {q}")
    print(f"    RAG-Answer > {rag_chain.invoke(q)}\n")
    print(f"    GPT-Answer > {llm.invoke(q).content}\n")
    

Query 1: Quem é o atual presidente da Argentina?
    RAG-Answer > O atual presidente da Argentina é Javier Milei, que assumiu o cargo em 10 de dezembro de 2023. Ele é um economista, político e professor argentino, líder do partido La Libertad Avanza. Milei foi eleito com 55,65% dos votos no segundo turno, tornando-se o presidente mais votado da história do país até então.
    GPT-Answer > O atual presidente da Argentina é Alberto Fernández. Ele assumiu o cargo em 10 de dezembro de 2019.

Query 2: Quem ganhou o Super Bowl 2024?
    RAG-Answer > Os Kansas City Chiefs ganharam o Super Bowl 2024, derrotando o San Francisco 49ers por 25-22 na prorrogação. Este foi o quarto título do Super Bowl da franquia e o terceiro em cinco anos. Patrick Mahomes foi fundamental na vitória, conectando-se com Mecole Hardman para o touchdown da vitória.
    GPT-Answer > Desculpe, mas não consigo fornecer informações em tempo real. Para descobrir quem ganhou o Super Bowl 2024, sugiro que você verifique os re

# Multiple PDF Queries

In [10]:
query = ["What were the valves tests?",
         "When was the report issued?",
         "what is the job number?",
         "What was the result of the inspection?",
         "What is the ITP reference number?",
         "What was the scope of the Sinopec project?",
         "How was the anti-static test performed?",
         "What is the standard of anti-static testing?"
         ]
for idx, q in enumerate(query):
    print(f"Query {idx+1}: {q}")
    print(f"    RAG-Answer > {rag_chain.invoke(q)}\n")
    print(f"    GPT-Answer > {llm.invoke(q).content}\n")

Query 1: What were the valves tests?
    RAG-Answer > The valve tests included witnessing pressure testing, torque testing, anti-static testing, visual inspection, and more. The pressure testing results were acceptable according to API 6D standards. The tests were conducted to ensure the valves met the required specifications.
    GPT-Answer > There are several tests that can be performed on valves to ensure they are functioning properly. Some common valve tests include:

1. Pressure testing: This involves subjecting the valve to high pressure to check for leaks or other defects.

2. Leakage testing: This test checks for any leakage of fluid through the valve when it is closed.

3. Flow testing: This test measures the flow rate of fluid through the valve to ensure it is within the specified range.

4. Functional testing: This test checks the overall functionality of the valve, including its ability to open and close properly.

5. Material testing: This test checks the material composit

# Queries

In [7]:
# Perguntas sobre Atualidades
actualities = ["Quem é o atual presidente da Argentina?",
               "Quais foram os feitos de Perri no Lyon?",
               "Quem é o jogador novo do Corinthians?",
               "Qual o grande problema do ATP Santiago?",
               ]

In [8]:
# Perguntas que demonstram capacidade de tradução
translation = ["Quem ganhou o Super Bowl 2024?",
               "Quanto foi o jogo PSG e Rennes?",
               "Qual é o escopo do projeto da Sinopec?"
               ]

In [9]:
# Perguntas com respostas objetivas
objective = ["Como foi realizado o teste antiestático?",
             "Qual é o padrão de teste antiestático?"
             ]

In [11]:
# make a list of lists
queries = [actualities, translation, objective]

# loop through the list of lists
for idx, q in enumerate(queries):
    if idx == 0:
        print(" ------ Perguntas sobre Atualidades ------ ")
    elif idx == 1:
        print(" ------ Perguntas que demonstram capacidade de tradução ------ ")
    else:
        print(" ------ Perguntas com respostas objetivas ------")
    for ix, i in enumerate(q):
        print(colored(f"Query {ix+1}: ", attrs=["bold"]) + colored(f"{i}", 'blue'))
        print(colored(f"    GPT(3.5-turbo)-Answer > ", attrs=["bold"]) + colored(f"{llm.invoke(i).content}", 'yellow'))
        print(colored(f"    RAG-Answer > ", attrs=["bold"]) + colored(f"{rag_chain_with_source.invoke(i)['answer']}", 'green'))
        for idx, doc in enumerate(rag_chain_with_source.invoke(i)["context"]):
            print(colored(f"        Document {idx+1}", attrs=["bold"]) + colored(f": {doc.metadata['title']}", 'blue'))
        print("\n")

 ------ Perguntas sobre Atualidades ------ 
Query 1: Quem é o atual presidente da Argentina?
    GPT(3.5-turbo)-Answer > O atual presidente da Argentina é Alberto Fernández. Ele assumiu o cargo em 10 de dezembro de 2019.
    RAG-Answer > O atual presidente da Argentina é Javier Milei, que assumiu o cargo em 10 de dezembro de 2023. Ele é um economista, político e professor argentino, líder do partido La Libertad Avanza. Milei foi eleito com 55,65% dos votos no segundo turno, tornando-se o presidente mais votado da história do país até então.
        Document 1: Javier Milei – Wikipédia, a enciclopédia livre
        Document 2: Javier Milei – Wikipédia, a enciclopédia livre
        Document 3: Javier Milei – Wikipédia, a enciclopédia livre
        Document 4: Javier Milei – Wikipédia, a enciclopédia livre


Query 2: Quais foram os feitos de Perri no Lyon?
    GPT(3.5-turbo)-Answer > Perri foi um jogador de futebol que atuou no Lyon durante a temporada 2000-2001. Durante sua passagem pelo

# Single Queries

In [12]:
q = "Qual o Kcon serial number for the gear operator?"
res = rag_chain_with_source.invoke(q)
print(colored(f"Query >", attrs=['bold']) + colored(f" {q}\n", 'blue'))
print(colored(f"GPT(3.5-turbo)-Answer > ", attrs=['bold']) + colored(f"{llm.invoke(q).content}\n", 'yellow'))
print(colored(f"RAG-Answer >", attrs=['bold']) + colored(f" {res['answer']}", 'green'))
for idx, doc in enumerate(res["context"]):
    print(colored(f"    Document {idx+1}", attrs=['bold']) + colored(f": {doc.metadata['title']}", 'blue'))


Query > Qual o Kcon serial number for the gear operator?

GPT(3.5-turbo)-Answer > O número de série do Kcon para o operador de engrenagens pode variar de acordo com o modelo específico do equipamento. Recomenda-se verificar o manual do usuário ou entrar em contato com o fabricante para obter informações precisas sobre o número de série do equipamento.

RAG-Answer > I don't know.
    Document 1: INSPECTION REPORT Nr LNP/BJ-12/048-CD01
    Document 2: INSPECTION REPORT Nr LNP/BJ-12/048-CD01
    Document 3: INSPECTION REPORT Nr LNP/BJ-12/048-CD01
    Document 4: INSPECTION REPORT Nr LNP/BJ-12/048-CD01
